In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy.stats import norm
from scipy import stats

#importing all usefull lib
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score, train_test_split, KFold, cross_val_predict
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
%matplotlib inline
from sklearn.linear_model import LinearRegression, RidgeCV, Lasso, ElasticNetCV, BayesianRidge, LassoLarsIC
from sklearn.metrics import mean_squared_error, make_scorer 
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
import math
from datetime import datetime
from scipy.stats import skew  # for some statistics
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from mlxtend.regressor import StackingCVRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import matplotlib.pyplot as plt
import scipy.stats as stats
import sklearn.linear_model as linear_model
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy.stats import norm
from scipy import stats

In [2]:
train_dataSet=pd.read_csv("/Users/chixinning/Desktop/kaggle/KaggleMyOwn/ReferenceCode/house-prices-advanced-regression-techniques/train.csv")
test_dataSet=pd.read_csv("/Users/chixinning/Desktop/kaggle/KaggleMyOwn/ReferenceCode/house-prices-advanced-regression-techniques/test.csv")

In [3]:
Sale_Price=train_dataSet.iloc[:,80]
Sale_Price.shape
train_dataSet=train_dataSet[train_dataSet.GrLivArea < 4500]
train_dataSet.reset_index(drop=True, inplace=True)
Sale_Price=Sale_Price.reset_index(drop=True)
train_dataSet.shape
test_dataSet.shape

(1459, 80)

In [4]:
train=train_dataSet.drop(["SalePrice"],axis=1)# combine test & train
train_data=pd.concat([train,test_dataSet],keys=['train_part','test_part'])
# train_data=pd.concat({'x':train,'y':test_dataSet})
X=train_data.loc["train_part"] # 这样就是runnable了
print(X.shape)
train_data.index=train_data["Id"]
train_data=train_data.drop(["Id"],axis=1)
# numerical -> categorical
train_data = train_data.replace({"MSSubClass" : {20 : "SC20", 30 : "SC30", 40 : "SC40", 45 : "SC45",50 : "SC50", 60 : "SC60", 70 : "SC70", 75 : "SC75", 80 : "SC80", 85 : "SC85", 90 : "SC90", 120 : "SC120", 150 : "SC150", 160 : "SC160", 180 : "SC180", 190 : "SC190"},"MoSold" : {1 : "Jan", 2 : "Feb", 3 : "Mar", 4 : "Apr", 5 : "May", 6 : "Jun",7 : "Jul", 8 : "Aug", 9 : "Sep", 10 : "Oct", 11 : "Nov", 12 : "Dec"}})  
train_data['YrSold'] = train_data['YrSold'].astype(str)# numerical -> categorical
train_data['MoSold'] = train_data['MoSold'].astype(str)# numerical -> categorical                      train_data['Functional'] = train_data['Functional'].fillna('Typ') #Home functionality (Assume typical unless deductions are warranted)
train_data['Exterior1st'] = train_data['Exterior1st'].fillna(train_data['Exterior1st'].mode()[0]) #Exterior covering on house
train_data['Exterior2nd'] = train_data['Exterior2nd'].fillna(train_data['Exterior2nd'].mode()[0])
# Exterior covering on house
train_data['SaleType'] = train_data['SaleType'].fillna(train_data['SaleType'].mode()[0])
#GarageYrBlt是因为没有车库还是缺失？
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    train_data[col] = train_data[col].fillna(0)
for col in ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']:
    train_data[col] = train_data[col].fillna('None')
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    train_data[col] = train_data[col].fillna('None')
# MSSubClass: Identifies the type of dwelling involved in the sale.	
# 将MSZoning和MSSubClass相关联
train_data['MSZoning'] = train_data.groupby('MSSubClass')['MSZoning'].transform(lambda x: x.fillna(x.mode()[0]))
objects = []
for i in train_data.columns:
    if train_data[i].dtype == object:
        objects.append(i)
train_data.update(train_data[objects].fillna('None'))
#LotFrontage: Linear feet of street connected to property
train_data['LotFrontage'] = train_data.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))

numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerics = []
for i in train_data.columns:
    if train_data[i].dtype in numeric_dtypes:
        numerics.append(i)
train_data.update(train_data[numerics].fillna(0))
num_col=train_data._get_numeric_data().columns.tolist()
cat_col=set(train_data.columns)-set(num_col)
print(len(num_col)+len(cat_col)) #未dummy前时刻check和是79
train_cat = train_data[cat_col]
pct=[]
for ix in train_cat.columns:
    temp=train_cat[ix].describe()
    # print(temp.describe)
    # break
    pct.append(temp["freq"]/temp["count"])
skewData=pd.DataFrame(pct,index=train_cat.columns,columns=["skewness"])
skewData=skewData.sort_values(by="skewness",ascending=False)
# print (skewData)
train_data = train_data.drop((skewData[skewData['skewness'] >= 0.95]).index,1) 
train_data.shape # 这里因为skewness行变了我怎么再找到test呢？
X=train_data.loc["train_part"] # 这样就是runnable了
print(X.shape)

(1458, 80)
79


KeyError: 'train_part'